In [54]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi3")


In [55]:
from langchain.embeddings import HuggingFaceEmbeddings

# Generate embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [56]:
from langchain.vectorstores import FAISS
# Load saved Vector Store
vectorstore = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

In [65]:
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate


retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 6,"score_threshold": 0.2})
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Only answer using the provided context. If the context is empty, just say that it is out of my scope If unsure, say 'I do not know.\n context: \n{context}'"),
    ("human", "question:\n {question}")
])

def custom_rag_qa(question):
    docs = retriever.get_relevant_documents(question)

    if not docs:  # If no relevant docs found
        return {"query": question, "result": "I cannot assist you with this query"}

    # If documents found, run through standard RAG
    rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type='stuff',
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)
    return rag_chain.invoke(question)




# Ask a question



In [66]:
result = custom_rag_qa("How to cook")
print(result)

c:\Users\kuber\anaconda3\envs\rag\Lib\site-packages\langchain_core\vectorstores\base.py:1080: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='d4bc54da-2e41-433d-8e8f-fd10967e0d9f', metadata={'title': 'nn.BatchNorm3d', 'page_links': ['https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm3d.html#batchnorm3d', 'https://pytorch.org/docs/stable/_modules/torch/nn/modules/batchnorm.html#BatchNorm3d', 'https://github.com/pytorch/pytorch/blob/v2.6.0/torch/nn/modules/batchnorm.py#L489', 'https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm3d.html#torch.nn.BatchNorm3d', 'https://arxiv.org/abs/1502.03167', 'https://docs.python.org/3/library/functions.html#int', 'https://docs.python.org/3/library/functions.html#float', 'https://docs.python.org/3/library/typing.html#typing.Optional', 'https://docs.python.org/3/library/functions.html#float', 'https://docs.python.org/3/library/functions.html#bool', 'https://docs.python.org/3/library/functions.html#bool', 'https

{'query': 'How to cook', 'result': 'I cannot assist you with this query'}


In [67]:
for i in result:
    print(i,result[i])

query How to cook
result I cannot assist you with this query
